In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Define data augmentation functions with less distortion
def augment_image(image):
    # Rotate the image by a random angle within a smaller range
    angle = np.random.randint(-20, 20)
    M = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1)
    rotated = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # Flip the image horizontally
    flipped = cv2.flip(rotated, 1)

    # Zoom into the image with a smaller range
    zoom_factor = np.random.uniform(0.9, 1.1)
    zoomed = cv2.resize(flipped, None, fx=zoom_factor, fy=zoom_factor)
    if zoom_factor < 1.0:
        delta_w = image.shape[1] - zoomed.shape[1]
        delta_h = image.shape[0] - zoomed.shape[0]
        zoomed = cv2.copyMakeBorder(zoomed, delta_h // 2, delta_h - delta_h // 2, delta_w // 2, delta_w - delta_w // 2, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    else:
        zoomed = cv2.resize(zoomed, (image.shape[1], image.shape[0]))

    # Brightness and contrast adjustment with smaller range
    brightness_factor = np.random.uniform(0.8, 1.2)
    brightness_adjusted = cv2.convertScaleAbs(zoomed, alpha=brightness_factor, beta=0)

    contrast_factor = np.random.uniform(0.8, 1.2)
    contrast_adjusted = cv2.convertScaleAbs(brightness_adjusted, alpha=contrast_factor, beta=0)

    # Random cropping with smaller range
    h, w, _ = contrast_adjusted.shape
    top = np.random.randint(0, h // 8)
    left = np.random.randint(0, w // 8)
    bottom = np.random.randint(7 * h // 8, h)
    right = np.random.randint(7 * w // 8, w)
    cropped = contrast_adjusted[top:bottom, left:right]
    cropped = cv2.resize(cropped, (150, 150))

    # Noise addition with smaller range
    noise = np.random.randn(150, 150, 3) * 5
    noisy_image = cropped + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return noisy_image

# Load the dataset and apply augmentation
image_exts = ['jpeg', 'jpg']
data_dir = 'data_'
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'

def load_images_from_folder(folder, augment=False, augment_times=3):
    images = []
    labels = []
    for image_class in os.listdir(folder):
        class_path = os.path.join(folder, image_class)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                if any(file_name.endswith(ext) for ext in image_exts):
                    file_path = os.path.join(class_path, file_name)
                    image = cv2.imread(file_path)
                    image = cv2.resize(image, (150, 150))  # Resize image to a fixed size
                    images.append(image)
                    labels.append(image_class)
                    if augment:
                        for _ in range(augment_times):
                            augmented_image = augment_image(image)
                            images.append(augmented_image)
                            labels.append(image_class)
    return np.array(images), np.array(labels)

X_train, y_train = load_images_from_folder(train_dir, augment=True, augment_times=3)
X_test, y_test = load_images_from_folder(test_dir, augment=False)

# HOG feature extraction
def extract_hog_features(image):
    return hog(image, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1), channel_axis=-1)

In [14]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Define data augmentation functions with less distortion
def augment_image(image):
    # Rotate the image by a random angle within a smaller range
    angle = np.random.randint(-20, 20)
    M = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1)
    rotated = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # Flip the image horizontally
    flipped = cv2.flip(rotated, 1)

    # Zoom into the image with a smaller range
    zoom_factor = np.random.uniform(0.9, 1.1)
    zoomed = cv2.resize(flipped, None, fx=zoom_factor, fy=zoom_factor)
    if zoom_factor < 1.0:
        delta_w = image.shape[1] - zoomed.shape[1]
        delta_h = image.shape[0] - zoomed.shape[0]
        zoomed = cv2.copyMakeBorder(zoomed, delta_h // 2, delta_h - delta_h // 2, delta_w // 2, delta_w - delta_w // 2, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    else:
        zoomed = cv2.resize(zoomed, (image.shape[1], image.shape[0]))

    # Brightness and contrast adjustment with smaller range
    brightness_factor = np.random.uniform(0.8, 1.2)
    brightness_adjusted = cv2.convertScaleAbs(zoomed, alpha=brightness_factor, beta=0)

    contrast_factor = np.random.uniform(0.8, 1.2)
    contrast_adjusted = cv2.convertScaleAbs(brightness_adjusted, alpha=contrast_factor, beta=0)

    # Random cropping with smaller range
    h, w, _ = contrast_adjusted.shape
    top = np.random.randint(0, h // 8)
    left = np.random.randint(0, w // 8)
    bottom = np.random.randint(7 * h // 8, h)
    right = np.random.randint(7 * w // 8, w)
    cropped = contrast_adjusted[top:bottom, left:right]
    cropped = cv2.resize(cropped, (150, 150))

    # Noise addition with smaller range
    noise = np.random.randn(150, 150, 3) * 5
    noisy_image = cropped + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return noisy_image

# Load the dataset and apply augmentation
image_exts = ['jpeg', 'jpg']
data_dir = 'data_'
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'

def load_images_from_folder(folder, augment=False, augment_times=3):
    images = []
    labels = []
    for image_class in os.listdir(folder):
        class_path = os.path.join(folder, image_class)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                if any(file_name.endswith(ext) for ext in image_exts):
                    file_path = os.path.join(class_path, file_name)
                    image = cv2.imread(file_path)
                    image = cv2.resize(image, (150, 150))  # Resize image to a fixed size
                    images.append(image)
                    labels.append(image_class)
                    if augment:
                        for _ in range(augment_times):
                            augmented_image = augment_image(image)
                            images.append(augmented_image)
                            labels.append(image_class)
    return np.array(images), np.array(labels)

X_train, y_train = load_images_from_folder(train_dir, augment=True, augment_times=3)
X_test, y_test = load_images_from_folder(test_dir, augment=False)

# HOG feature extraction
def extract_hog_features(image):
    return hog(image, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1), channel_axis=-1)

X_train_hog = np.array([extract_hog_features(image) for image in X_train])
X_test_hog = np.array([extract_hog_features(image) for image in X_test])

# Flatten the HOG features
X_train_hog = X_train_hog.reshape(X_train_hog.shape[0], -1)
X_test_hog = X_test_hog.reshape(X_test_hog.shape[0], -1)

# Normalize the features
scaler = StandardScaler()
X_train_hog = scaler.fit_transform(X_train_hog)
X_test_hog = scaler.transform(X_test_hog)

# Define the parameter grid for GridSearchCV with a wider range
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

# Create a GridSearchCV object
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2, cv=5, n_jobs=-1)
grid.fit(X_train_hog, y_train)

# Get the best parameters from the grid search
best_params = grid.best_params_
print(f"Best parameters found: {best_params}")

# Build the SVM model with the best parameters
best_model = SVC(C=best_params['C'], gamma=best_params['gamma'], kernel=best_params['kernel'])
best_model.fit(X_train_hog, y_train)

# Predict on the test set
y_pred = best_model.predict(X_test_hog)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters found: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Accuracy: 41.85%
              precision    recall  f1-score   support

  basketball       0.40      0.80      0.53        56
   bike polo       0.00      0.00      0.00        15
   billiards       0.43      0.74      0.55        35
    football       0.00      0.00      0.00        29
    swimming       0.67      0.22      0.33        18
  volleyball       0.67      0.06      0.12        31

    accuracy                           0.42       184
   macro avg       0.36      0.31      0.26       184
weighted avg       0.38      0.42      0.32       184



C:\Users\thanh.le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thanh.le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thanh.le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificatio

In [18]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load the dataset without augmentation
image_exts = ['jpeg', 'jpg']
data_dir = 'data_'
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'

def load_images_from_folder(folder):
    images = []
    labels = []
    for image_class in os.listdir(folder):
        class_path = os.path.join(folder, image_class)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                if any(file_name.endswith(ext) for ext in image_exts):
                    file_path = os.path.join(class_path, file_name)
                    image = cv2.imread(file_path)
                    image = cv2.resize(image, (150, 150))  # Resize image to a fixed size
                    images.append(image)
                    labels.append(image_class)
    return np.array(images), np.array(labels)

X_train, y_train = load_images_from_folder(train_dir)
X_test, y_test = load_images_from_folder(test_dir)

# HOG feature extraction
def extract_hog_features(image):
    return hog(image, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1), channel_axis=-1)

X_train_hog = np.array([extract_hog_features(image) for image in X_train])
X_test_hog = np.array([extract_hog_features(image) for image in X_test])

# Flatten the HOG features
X_train_hog = X_train_hog.reshape(X_train_hog.shape[0], -1)
X_test_hog = X_test_hog.reshape(X_test_hog.shape[0], -1)

# Normalize the features
scaler = StandardScaler()
X_train_hog = scaler.fit_transform(X_train_hog)
X_test_hog = scaler.transform(X_test_hog)

# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

# Create a GridSearchCV object
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2, cv=5, n_jobs=-1)
grid.fit(X_train_hog, y_train)

# Get the best parameters from the grid search
best_params = grid.best_params_
print(f"Best parameters found: {best_params}")

# Build the SVM model with the best parameters
best_model = SVC(C=best_params['C'], gamma=best_params['gamma'], kernel=best_params['kernel'])
best_model.fit(X_train_hog, y_train)

# Predict on the test set
y_pred = best_model.predict(X_test_hog)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters found: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Accuracy: 42.39%
              precision    recall  f1-score   support

  basketball       0.36      0.96      0.52        56
   bike polo       0.00      0.00      0.00        15
   billiards       0.92      0.31      0.47        35
    football       0.33      0.03      0.06        29
    swimming       0.65      0.61      0.63        18
  volleyball       1.00      0.03      0.06        31

    accuracy                           0.42       184
   macro avg       0.54      0.33      0.29       184
weighted avg       0.57      0.42      0.33       184



C:\Users\thanh.le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thanh.le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thanh.le\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificatio

In [41]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

# Define data augmentation functions
def augment_image(image):
    # Rotate the image by a random angle
    angle = np.random.randint(-40, 40)
    M = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1)
    rotated = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # Flip the image horizontally
    flipped = cv2.flip(rotated, 1)

    # Zoom into the image
    zoom_factor = np.random.uniform(0.8, 1.2)
    zoomed = cv2.resize(flipped, None, fx=zoom_factor, fy=zoom_factor)
    if zoom_factor < 1.0:
        delta_w = image.shape[1] - zoomed.shape[1]
        delta_h = image.shape[0] - zoomed.shape[0]
        zoomed = cv2.copyMakeBorder(zoomed, delta_h // 2, delta_h - delta_h // 2, delta_w // 2, delta_w - delta_w // 2, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    else:
        zoomed = cv2.resize(zoomed, (image.shape[1], image.shape[0]))

    # Brightness adjustment
    brightness_factor = np.random.uniform(0.5, 1.5)
    brightness_adjusted = cv2.convertScaleAbs(zoomed, alpha=brightness_factor, beta=0)

    # Contrast adjustment
    contrast_factor = np.random.uniform(0.5, 1.5)
    contrast_adjusted = cv2.convertScaleAbs(brightness_adjusted, alpha=contrast_factor, beta=0)

    # Random cropping
    h, w, _ = contrast_adjusted.shape
    top = np.random.randint(0, h // 4)
    left = np.random.randint(0, w // 4)
    bottom = np.random.randint(3 * h // 4, h)
    right = np.random.randint(3 * w // 4, w)
    cropped = contrast_adjusted[top:bottom, left:right]
    cropped = cv2.resize(cropped, (150, 150))

    # Noise addition
    noise = np.random.randn(150, 150, 3) * 10
    noisy_image = cropped + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return noisy_image

# Load the dataset and apply augmentation
image_exts = ['jpeg', 'jpg']
data_dir = 'data_'
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'

def load_images_from_folder(folder):
    images = []
    labels = []
    for image_class in os.listdir(folder):
        class_path = os.path.join(folder, image_class)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                if any(file_name.endswith(ext) for ext in image_exts):
                    file_path = os.path.join(class_path, file_name)
                    image = cv2.imread(file_path)
                    image = cv2.resize(image, (150, 150))  # Resize image to a fixed size
                    images.append(image)
                    labels.append(image_class)
    return np.array(images), np.array(labels)

X_train, y_train = load_images_from_folder(train_dir)
X_test, y_test = load_images_from_folder(test_dir)

# HOG feature extraction
def extract_hog_features(image):
    return hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), channel_axis=-1)

X_train_hog = np.array([extract_hog_features(image) for image in X_train])
X_test_hog = np.array([extract_hog_features(image) for image in X_test])

# Flatten the HOG features
X_train_hog = X_train_hog.reshape(X_train_hog.shape[0], -1)
X_test_hog = X_test_hog.reshape(X_test_hog.shape[0], -1)

# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf','linear','poly','sigmoid']
}

# Create a GridSearchCV object
grid = GridSearchCV(SVC(), param_grid, 
                    refit=True, verbose=2, 
                    cv=2, n_jobs=-1)
grid.fit(X_train_hog, y_train)

# Get the best parameters from the grid search
best_params = grid.best_params_
print(f"Best parameters found: {best_params}")

# Build the SVM model with the best parameters
best_model = SVC(
                probability=True,
                C=best_params['C'], 
                gamma=best_params['gamma'], 
                kernel=best_params['kernel'])
best_model.fit(X_train_hog, y_train)

# Predict on the test set
y_pred = best_model.predict(X_test_hog)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1_score = classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score']
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'F1 Score: {f1_score * 100:.2f}%')
print(classification_report(y_test, y_pred))


Fitting 2 folds for each of 64 candidates, totalling 128 fits
Best parameters found: {'C': 0.1, 'gamma': 0.01, 'kernel': 'poly'}
Accuracy: 53.80%
F1 Score: 52.95%
              precision    recall  f1-score   support

  basketball       0.47      0.66      0.55        56
   bike polo       1.00      0.20      0.33        15
   billiards       0.75      0.69      0.72        35
    football       0.52      0.45      0.48        29
    swimming       0.58      0.61      0.59        18
  volleyball       0.41      0.35      0.38        31

    accuracy                           0.54       184
   macro avg       0.62      0.49      0.51       184
weighted avg       0.58      0.54      0.53       184



In [27]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Define data augmentation function
def augment_image(image):
    # Rotate the image by a random angle
    angle = np.random.randint(-20, 20)
    M = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1)
    rotated = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # Flip the image horizontally
    flipped = cv2.flip(rotated, 1)

    # Zoom into the image
    zoom_factor = np.random.uniform(0.9, 1.1)
    zoomed = cv2.resize(flipped, None, fx=zoom_factor, fy=zoom_factor)
    if zoom_factor < 1.0:
        delta_w = image.shape[1] - zoomed.shape[1]
        delta_h = image.shape[0] - zoomed.shape[0]
        zoomed = cv2.copyMakeBorder(zoomed, delta_h // 2, delta_h - delta_h // 2, delta_w // 2, delta_w - delta_w // 2, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    else:
        zoomed = cv2.resize(zoomed, (image.shape[1], image.shape[0]))

    # Brightness adjustment
    brightness_factor = np.random.uniform(0.8, 1.2)
    brightness_adjusted = cv2.convertScaleAbs(zoomed, alpha=brightness_factor, beta=0)

    # Contrast adjustment
    contrast_factor = np.random.uniform(0.8, 1.2)
    contrast_adjusted = cv2.convertScaleAbs(brightness_adjusted, alpha=contrast_factor, beta=0)

    # Random cropping
    h, w, _ = contrast_adjusted.shape
    top = np.random.randint(0, h // 8)
    left = np.random.randint(0, w // 8)
    bottom = np.random.randint(7 * h // 8, h)
    right = np.random.randint(7 * w // 8, w)
    cropped = contrast_adjusted[top:bottom, left:right]
    cropped = cv2.resize(cropped, (150, 150))

    # Noise addition
    noise = np.random.randn(150, 150, 3) * 5
    noisy_image = cropped + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return noisy_image

# Load the dataset with augmentation
def load_images_from_folder(folder, augment=False, augment_times=3):
    images = []
    labels = []
    for image_class in os.listdir(folder):
        class_path = os.path.join(folder, image_class)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                if any(file_name.endswith(ext) for ext in ['jpeg', 'jpg']):
                    file_path = os.path.join(class_path, file_name)
                    image = cv2.imread(file_path)
                    image = cv2.resize(image, (150, 150))  # Resize image to a fixed size
                    images.append(image)
                    labels.append(image_class)
                    if augment:
                        for _ in range(augment_times):
                            augmented_image = augment_image(image)
                            images.append(augmented_image)
                            labels.append(image_class)
    return np.array(images), np.array(labels)

# Load datasets
X_train, y_train = load_images_from_folder(train_dir, augment=True, augment_times=2)
X_test, y_test = load_images_from_folder(test_dir, augment=False)

# HOG feature extraction with adjusted parameters
def extract_hog_features(image):
    return hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), channel_axis=-1)

X_train_hog = np.array([extract_hog_features(image) for image in X_train])
X_test_hog = np.array([extract_hog_features(image) for image in X_test])

# Flatten the HOG features
X_train_hog = X_train_hog.reshape(X_train_hog.shape[0], -1)
X_test_hog = X_test_hog.reshape(X_test_hog.shape[0], -1)

# Define a reduced parameter grid for GridSearchCV
param_grid = {
    'C': [1, 10, 100],
    'gamma': [0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

# Create a GridSearchCV object with fewer cross-validation folds
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2, cv=3, n_jobs=-1)
grid.fit(X_train_hog, y_train)

# Get the best parameters from the grid search
best_params = grid.best_params_
print(f"Best parameters found: {best_params}")

# Build the SVM model with the best parameters
best_model = SVC(C=best_params['C'], gamma=best_params['gamma'], kernel=best_params['kernel'])
best_model.fit(X_train_hog, y_train)

# Predict on the test set
y_pred = best_model.predict(X_test_hog)

from sklearn.metrics import f1_score
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1_score = f1_score(y_test, y_pred, average='weighted')
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'F1 Score: {f1_score * 100:.2f}%')
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 9 candidates, totalling 27 fits
Best parameters found: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Accuracy: 53.26%
              precision    recall  f1-score   support

  basketball       0.49      0.70      0.58        56
   bike polo       1.00      0.33      0.50        15
   billiards       0.57      0.80      0.67        35
    football       0.53      0.31      0.39        29
    swimming       0.78      0.39      0.52        18
  volleyball       0.40      0.32      0.36        31

    accuracy                           0.53       184
   macro avg       0.63      0.48      0.50       184
weighted avg       0.57      0.53      0.52       184



In [30]:
#import f1_score from sklearn.metrics
from sklearn.metrics import f1_score
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1_score = f1_score(y_test, y_pred, average='weighted')
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'F1 Score: {f1_score * 100:.2f}%')
print(classification_report(y_test, y_pred))

Accuracy: 53.26%
F1 Score: 51.60%
              precision    recall  f1-score   support

  basketball       0.49      0.70      0.58        56
   bike polo       1.00      0.33      0.50        15
   billiards       0.57      0.80      0.67        35
    football       0.53      0.31      0.39        29
    swimming       0.78      0.39      0.52        18
  volleyball       0.40      0.32      0.36        31

    accuracy                           0.53       184
   macro avg       0.63      0.48      0.50       184
weighted avg       0.57      0.53      0.52       184



In [38]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset without augmentation
def load_images_from_folder(folder):
    images = []
    labels = []
    for image_class in os.listdir(folder):
        class_path = os.path.join(folder, image_class)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                if any(file_name.endswith(ext) for ext in ['jpeg', 'jpg']):
                    file_path = os.path.join(class_path, file_name)
                    image = cv2.imread(file_path)
                    image = cv2.resize(image, (150, 150))  # Resize image to a fixed size
                    images.append(image)
                    labels.append(image_class)
    return np.array(images), np.array(labels)

X_train, y_train = load_images_from_folder(train_dir)
X_test, y_test = load_images_from_folder(test_dir)

# HOG feature extraction
def extract_hog_features(image):
    return hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), channel_axis=-1)

X_train_hog = np.array([extract_hog_features(image) for image in X_train])
X_test_hog = np.array([extract_hog_features(image) for image in X_test])

# Flatten the HOG features
X_train_hog = X_train_hog.reshape(X_train_hog.shape[0], -1)
X_test_hog = X_test_hog.reshape(X_test_hog.shape[0], -1)

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_hog, y_train)

# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

# Create a GridSearchCV object with more cross-validation folds
grid = GridSearchCV(SVC(probability=True), param_grid, refit=True, verbose=2, cv=3, n_jobs=-1)
grid.fit(X_train_resampled, y_train_resampled)

# Get the best parameters from the grid search
best_params = grid.best_params_
print(f"Best parameters found: {best_params}")

# Build the SVM model with the best parameters
best_model = SVC(C=best_params['C'], gamma=best_params['gamma'], kernel=best_params['kernel'])
best_model.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred = best_model.predict(X_test_hog)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1_score = classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score']
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'F1 Score: {f1_score * 100:.2f}%')
print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best parameters found: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
Accuracy: 55.43%
F1 Score: 55.10%
              precision    recall  f1-score   support

  basketball       0.53      0.55      0.54        56
   bike polo       0.71      0.33      0.45        15
   billiards       0.69      0.71      0.70        35
    football       0.49      0.59      0.53        29
    swimming       0.55      0.61      0.58        18
  volleyball       0.48      0.42      0.45        31

    accuracy                           0.55       184
   macro avg       0.58      0.54      0.54       184
weighted avg       0.56      0.55      0.55       184



In [42]:
# save the model
import joblib
joblib.dump(best_model, 'best_model.pkl')

['best_model.pkl']

In [35]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1_score = classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score']
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'F1 Score: {f1_score * 100:.2f}%')
print(classification_report(y_test, y_pred))

Accuracy: 54.89%
F1 Score: 54.49%
              precision    recall  f1-score   support

  basketball       0.51      0.55      0.53        56
   bike polo       0.71      0.33      0.45        15
   billiards       0.69      0.71      0.70        35
    football       0.49      0.59      0.53        29
    swimming       0.55      0.61      0.58        18
  volleyball       0.48      0.39      0.43        31

    accuracy                           0.55       184
   macro avg       0.57      0.53      0.54       184
weighted avg       0.56      0.55      0.54       184



In [43]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

# Define data augmentation functions
def augment_image(image):
    # Rotate the image by a random angle
    angle = np.random.randint(-40, 40)
    M = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1)
    rotated = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # Flip the image horizontally
    flipped = cv2.flip(rotated, 1)

    # Zoom into the image
    zoom_factor = np.random.uniform(0.8, 1.2)
    zoomed = cv2.resize(flipped, None, fx=zoom_factor, fy=zoom_factor)
    if zoom_factor < 1.0:
        delta_w = image.shape[1] - zoomed.shape[1]
        delta_h = image.shape[0] - zoomed.shape[0]
        zoomed = cv2.copyMakeBorder(zoomed, delta_h // 2, delta_h - delta_h // 2, delta_w // 2, delta_w - delta_w // 2, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    else:
        zoomed = cv2.resize(zoomed, (image.shape[1], image.shape[0]))

    # Brightness adjustment
    brightness_factor = np.random.uniform(0.5, 1.5)
    brightness_adjusted = cv2.convertScaleAbs(zoomed, alpha=brightness_factor, beta=0)

    # Contrast adjustment
    contrast_factor = np.random.uniform(0.5, 1.5)
    contrast_adjusted = cv2.convertScaleAbs(brightness_adjusted, alpha=contrast_factor, beta=0)

    # Random cropping
    h, w, _ = contrast_adjusted.shape
    top = np.random.randint(0, h // 4)
    left = np.random.randint(0, w // 4)
    bottom = np.random.randint(3 * h // 4, h)
    right = np.random.randint(3 * w // 4, w)
    cropped = contrast_adjusted[top:bottom, left:right]
    cropped = cv2.resize(cropped, (150, 150))

    # Noise addition
    noise = np.random.randn(150, 150, 3) * 10
    noisy_image = cropped + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return noisy_image

# Load the dataset and apply augmentation
image_exts = ['jpeg', 'jpg']
data_dir = 'data_'
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'

def load_images_from_folder(folder):
    images = []
    labels = []
    for image_class in os.listdir(folder):
        class_path = os.path.join(folder, image_class)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                if any(file_name.endswith(ext) for ext in image_exts):
                    file_path = os.path.join(class_path, file_name)
                    image = cv2.imread(file_path)
                    image = cv2.resize(image, (150, 150))  # Resize image to a fixed size
                    images.append(image)
                    labels.append(image_class)
    return np.array(images), np.array(labels)

X_train, y_train = load_images_from_folder(train_dir)
X_test, y_test = load_images_from_folder(test_dir)

# HOG feature extraction
def extract_hog_features(image):
    return hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), channel_axis=-1)

X_train_hog = np.array([extract_hog_features(image) for image in X_train])
X_test_hog = np.array([extract_hog_features(image) for image in X_test])

# Flatten the HOG features
X_train_hog = X_train_hog.reshape(X_train_hog.shape[0], -1)
X_test_hog = X_test_hog.reshape(X_test_hog.shape[0], -1)

# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf','linear','poly','sigmoid']
}

# Create a GridSearchCV object
grid = GridSearchCV(SVC(), param_grid, 
                    refit=True, verbose=2, 
                    cv=2, n_jobs=-1)
grid.fit(X_train_hog, y_train)

# Get the best parameters from the grid search
best_params = grid.best_params_
print(f"Best parameters found: {best_params}")

# Build the SVM model with the best parameters
best_model = SVC(
                probability=True,
                C=best_params['C'], 
                gamma=best_params['gamma'], 
                kernel=best_params['kernel'])
best_model.fit(X_train_hog, y_train)

# Predict on the test set
y_pred = best_model.predict(X_test_hog)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1_score = classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score']
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'F1 Score: {f1_score * 100:.2f}%')
print(classification_report(y_test, y_pred))

# save the model
import joblib
joblib.dump(best_model, 'best_model1.pkl')

Fitting 2 folds for each of 64 candidates, totalling 128 fits
Best parameters found: {'C': 0.1, 'gamma': 0.01, 'kernel': 'poly'}
Accuracy: 53.80%
F1 Score: 52.95%
              precision    recall  f1-score   support

  basketball       0.47      0.66      0.55        56
   bike polo       1.00      0.20      0.33        15
   billiards       0.75      0.69      0.72        35
    football       0.52      0.45      0.48        29
    swimming       0.58      0.61      0.59        18
  volleyball       0.41      0.35      0.38        31

    accuracy                           0.54       184
   macro avg       0.62      0.49      0.51       184
weighted avg       0.58      0.54      0.53       184



['best_model1.pkl']